In [9]:
# Convolutional Neural Network
# Image Recognition / Image Classification Problem - Train a CNN to Predict if image is of Dog or Cat
# This model can be applied to predict if person has tumor or not

In [10]:
# PART 0 - Data PreProcessesing
# Used Keras as Library with TensorFlow as backened
# Total Images=10k jpeg, Train_Set=8k, Test_Set=2k [Train_Dog=4k, Train_Cat=4k, Test_Dog=1k, Test_Cat=1k]
# Actual Kaggle Dataset:25k images

In [11]:
# Dataset : we dont have a csv file. To use KERAS, to import the images, we have to create a proper folder structure.
# Separate Images into Train and Test. Within Train, create 2 folders, one for cat and one folder for dog. This is how KERAS
# will differentiate the labels of ur dependent variable

In [12]:
# PART 1 - Building the CNN

In [13]:
# Importing the Keras libraries and packages
from keras.models import Sequential # used to initialize the Convolutional Neural Network
from keras.layers import Conv2D # for adding Convolutional Layer
from keras.layers import MaxPooling2D # for adding Pooling Layer
from keras.layers import Flatten # to flatten the Poolng Layer
from keras.layers import Dense # to add the full connected layer in ANN

In [14]:
# Initialising the CNN
classifier = Sequential() # created an object named classifier of Sequential class

In [15]:
# Step 1 - Convolution
classifier.add(Conv2D(32, (3, 3), input_shape = (128, 128, 3), activation = 'relu')) # Add-Method, Conv2D-Function, Pass Arguments

# nb_filter=no of feature detectors or filters. Common practice is start w 32 and then we can add 64, 128, 256
# And no of rows and columns in the feature detector
# Conv2D(32, (3, 3) means that we would create 32 feature detectors, each of 3 by 3 dimensions and hence our convolutional
# layer will be composed of 32 feature maps

# input_shape = (64, 64, 3) = is shape of input image. Expected size of image for CNN model.
# I/p images are converted into 3D array if the images are coloured. 
# 3D array is composed of 3 channels. Each channel corresponds to one 2D array, that contains pixels of the image
# 3 is no of channels, since this is a coloured image - red, blue and green
# 64 and 64 are dimensions of 2D array in each of the channels

# activation = 'relu' - We might get -ve pixels in the feature maps after convolution operation. We nedd to get rid of these
# -ve pixels, to have the non linearity in the CNN

In [16]:
# Step 2 - Pooling
classifier.add(MaxPooling2D(pool_size = (2, 2))) # Pooling consists of reducing the size of the feature maps (used stride of 2)
# Helps in reducing the number of nodes that will go into ANN, so that its less computationally expensive

In [17]:
# Adding a second convolutional layer
classifier.add(Conv2D(32, (3, 3), activation = 'relu')) # we dont need to input_shape = (64, 64, 3) paramter, bcz CNN will 
# automatically take input from pooled feature map
classifier.add(MaxPooling2D(pool_size = (2, 2)))

# Adding a third convolutional layer
classifier.add(Conv2D(64, (3, 3), activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))

In [18]:
# Step 3 - Flattening
classifier.add(Flatten()) # Flattening means taking all the pooled feature maps and put them in a single vector

In [19]:
# Step 4 - Full connection

# Adding Fully conncted layer
classifier.add(Dense(units = 128, activation = 'relu')) 
# units = 128 (Common Practice : No of nodes in hiddel layer = avg of no of nodes in i/p layer and o/p layer)
# But since here we have lots of i/p nodes, by experiment, we know that number around 100 is good, so we use 128. Its
# good practice to choose a no as power of 2
#activation = 'relu' # Hiddel layer activation function

# Add output layer
classifier.add(Dense(units = 1, activation = 'sigmoid')) # units = 1, no of nodes in output layer
# activation = 'sigmoid' - sigmoid gives binary outcome, relu gives probability as output

In [20]:
# Compiling the CNN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [21]:
# Part 2 - Fitting the CNN to the images
# We finished building the CNN architecture. Now we will fit this CNN to the Images

# Image Augmnetation: is a technique to enrich our train dataset, w/o adding more images. Process of preprocessesing the images 
# to prevent overfitting (Great result on train & poor result on test). Amout of training images is augumented.
# One of the situation that leads to overfitting is when we have data to train the model. In such case, model is able to find 
# the correlations on train data but fail to generalize the correlations on new or test data.
# In case of image, we need to have lot of images to find the correlations, patterns etc. We are working with only 10K images.
# Either we can use more images or use a trick called Image Augmnetation, where it will create many batches of the images
# and in each batch, it will apply some random transformation on random selection of images like rotating, flipping, 
# shifting them etc
# So we would get many more diverse images in train set to train the model

# Image Augmnetation is applied only on training set
from keras.preprocessing.image import ImageDataGenerator

# Create an object of ImageDataGenerator class
train_datagen = ImageDataGenerator(rescale = 1./255, # Pixels take values b/w 0 & 255. So we rescale them b/w 0 and 1
                                   shear_range = 0.2, # apply transformations on image 
                                   zoom_range = 0.2, # apply some random zooms
                                   horizontal_flip = True) # images will flip horizontly

test_datagen = ImageDataGenerator(rescale = 1./255) # Here just rescale the test images b/w 0 and 1

In [22]:
# Creates Training Set
# Also applied ImageDataGenerator on train set
training_set = train_datagen.flow_from_directory('training_set', # dataset directory
                                                 target_size = (128, 128), # expected size of image
                                                 batch_size = 32, # 32 images will be randomly selected & passed through CNN
                                                                  # after which the weights would be updated
                                                 class_mode = 'binary') # indicates DV is binary
# Creates Test Set
test_set = test_datagen.flow_from_directory('test_set',
                                            target_size = (128, 128),
                                            batch_size = 32,
                                            class_mode = 'binary')

Found 8000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [23]:
# Fit CNN to training set & also vaildating the performance on test set
classifier.fit_generator(training_set,
                         steps_per_epoch = 8000, # No of images
                         epochs = 1, #25
                         validation_data = test_set,
                         validation_steps = 2000)

# Accuracy - Train : 84%, Test :75% (Now in order to improve the accuracy on the Test set, we would add 2nd Convolutional layer
# after Step 2 of Pooling)

# After adding 2nd Convolutional layer
# Accuracy - Train : 85%, Test :82%

# Changed input_shape = (64, 64, 3) to (128,128,3). Added 3rd Convolutional layer w 64 Feature detector instead of 32
# Accuracy - Train : 92%, Test :90%

# 
# Adding a third convolutional layer
classifier.add(Conv2D(64, (3, 3), activation='relu'))

# How to improve Accuracy?
# - Adding set of concolutional layers
# - Add more images to your dataset
# - Increase the no of feature detectors from 32, 64, 128, 256 [classifier.add(Conv2D(32, (3, 3), activation = 'relu'))]
# - Choose higher traget size for images of your training set and test set, so that u get more information of ur pixel patterns
#   target_size = (64, 64) Increase this size

Epoch 1/1
8000/8000 [==============================] - 10632s 1s/step - loss: 0.2980 - acc: 0.8620 - val_loss: 0.5582 - val_acc: 0.8341


In [27]:
# Part 3 - Making new predictions

import numpy as np
from keras.preprocessing import image
test_image = image.load_img('single_prediction/cat_or_dog_2.jpg', target_size = (128, 128)) # creates arry of 2D
test_image = image.img_to_array(test_image) # creates arry of 3D
test_image = np.expand_dims(test_image, axis = 0) # predict function expects 4D instead of 3D, so we use expand_dims
result = classifier.predict(test_image)
training_set.class_indices
if result[0][0] == 1:
    prediction = 'dog'
else:
    prediction = 'cat'

In [28]:
prediction

'cat'